## IMPORTS 

In [ ]:
!pip install bs4

In [ ]:
!pip install ktrain

In [ ]:
!pip install pytorch-nlp

In [ ]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

from string import punctuation
import string
from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import bs4
from bs4 import BeautifulSoup
import re
from torch.autograd import Variable

import matplotlib.pyplot as plt
import pickle
from datetime import datetime

### Basic Parameters

In [ ]:
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 46 # max number of words in a question to use 750
batch_size = 46 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits
SEED = 10
debug = 0

In [ ]:
# new dataframe
train_df=pd.read_csv("../input/sentiment-analysis-for-financial-news/all-data.csv", sep=',', encoding='latin-1',names = ["category","comment"])
train_df['comment'].fillna('',inplace=True)
train_df['comment'] = train_df['comment'].apply(lambda x: x.strip())
train_df['category']=train_df['category'].replace(to_replace="neutral",value=0)
train_df['category']=train_df['category'].replace(to_replace="negative",value=1)
train_df['category']=train_df['category'].replace(to_replace="positive",value=2)

print("neutral:", (train_df ['category']== 0).astype(int).sum(axis=0))
print("negative:",(train_df ['category']== 1).astype(int).sum(axis=0))
print("positive:",(train_df ['category']== 2).astype(int).sum(axis=0))

In [ ]:
class CleanText:
    def __init__(self,review):
        self.review=review.lower()
    
    def remove_URL(self):
        url = re.compile(r'https?://\S+|www\.\S+')
    
        return url.sub('', self.review)
    
    def remove_html(self):
        html = re.compile(r'<.*?>')
    
        return html.sub('', self.review)
    
    def remove_not_ASCII(self):
        
        review = ''.join([word for word in self.review if word in string.printable])
        return review
    
    def remove_emoji(self):
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
        return emoji_pattern.sub(r'', self.review)
    
    
    def remove_punct(self):
        table = str.maketrans('', '', string.punctuation)
    
        return self.review.translate(table)
    
    def remove_numbers(self):

#         t = ''.join([i for i in self.review if not i.isdigit()])
        num = re.compile(r'[-+]?[.\d]*[\d]+[:,.\d]*')
        return num.sub(r'NUMBER', self.review)

#         return t
    
    
    def review_to_words(self):
        nltk.download("stopwords", quiet=True)
        stemmer = PorterStemmer()
        
        words = BeautifulSoup(self.review, "html.parser").get_text() # Remove HTML tags
#         words = re.sub(r"[^a-zA-Z0-9]", " ", self.review.lower()) # Convert to lower case
        
#         words=self.review.lower()
        words=self.review.split() # Split string into words
        words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
        words = [PorterStemmer().stem(w) for w in words] # stem
        
        return words
    
    def all_func(self):
        self.review=self.remove_URL()
        self.review=self.remove_html()
        self.review=self.remove_not_ASCII()
        self.review=self.remove_emoji()
        self.review=self.remove_punct()
        self.review=self.remove_numbers()
        self.review=self.review_to_words()
        
        return self.review

In [ ]:
train_df['comment']=train_df['comment'].apply(lambda x: CleanText(x).all_func())

## Finding the maxlen

In [ ]:
train_df['len'] = train_df['comment'].apply(lambda s : len(s))

In [ ]:
train_df['len'].plot.hist(bins=100)

## Preprocessing Y Column
We are only going to be classifying conditions for which the count of reviews are more than 3000.

In [ ]:
count_df1 = train_df[['category','comment']].groupby('category').aggregate({'comment':'count'}).reset_index().sort_values('comment',ascending=False)
count_df1.head()

In [ ]:
target_conditions1 = count_df1[count_df1['comment']>1]['category'].values
target_conditions1 

In [ ]:
# import re

# def clean_text(x):
#     pattern = r'[^a-zA-z0-9\s]'
#     text = re.sub(pattern, '', x)
#     return x

# def clean_numbers(x):
#     if bool(re.search(r'\d', x)):
#         x = re.sub('[0-9]{5,}', '#####', x)
#         x = re.sub('[0-9]{4}', '####', x)
#         x = re.sub('[0-9]{3}', '###', x)
#         x = re.sub('[0-9]{2}', '##', x)
#     return x

In [ ]:
# contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
# def _get_contractions(contraction_dict):
#     contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
#     return contraction_dict, contraction_re
# contractions, contractions_re = _get_contractions(contraction_dict)
# def replace_contractions(text):
#     def replace(match):
#         return contractions[match.group(0)]
#     return contractions_re.sub(replace, text)
# # Usage
# replace_contractions("this's a text with contraction")

In [ ]:
train_df['category'].unique()

In [ ]:
def drop_empty_space(df,column_name):
    for ind,el in enumerate(df[column_name]):
        
        if not el:
            df.drop(ind,inplace=True)
            
    return df

In [ ]:
train_df=drop_empty_space(train_df,'comment')

In [ ]:
train_df

In [ ]:
len(train_df)

In [ ]:
# my data
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(train_df['comment'], train_df['category'],
                                                    stratify=train_df['category'], 
                                                    test_size=0.25)

In [ ]:
# train_X.to_csv('./trainx.csv',index=False)
# len(train_X)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_y = le.fit_transform(train_y.values)
test_y = le.transform(test_y.values)
print(len(train_y))
print(len(test_y))

In [ ]:
def convert_and_pad(word_dict, sentence, pad=46):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad

    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:      
            working_sentence[word_index+pad-len(sentence)] = word_dict[word]                      
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=46):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)


def words_to_numbers(x_train):
    words=[w for s in x_train for w in s]
    counts=Counter(words)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word:ii for ii,word in enumerate(vocab,1)}
    
    return vocab_to_int

def words_as_int(vocab_to_int,x_train):
    news_int = []
    x=0
    for review in x_train:
        x+=1
        news_int.append([vocab_to_int[word] for word in review])
        
    return news_int

In [ ]:
vocab_to_int_train_x=words_to_numbers(train_X)


In [ ]:
len(vocab_to_int_train_x)

In [ ]:
words_as_int_train_x=words_as_int(vocab_to_int_train_x,train_X)
len(words_as_int_train_x)

In [ ]:
# Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.
news_len_train = Counter([len(x) for x in words_as_int_train_x])
print("Zero-length reviews: {}".format(news_len_train[0]))
print("Maximum review length: {}".format(max(news_len_train)))

In [ ]:
padded_train_X,_=convert_and_pad_data(vocab_to_int_train_x, train_X, pad=46)

In [ ]:
padded_train_X.shape

In [ ]:
padded_train_X=padded_train_X.astype('int32')

In [ ]:
padded_test_X,_=convert_and_pad_data(vocab_to_int_train_x, test_X, pad=46)
padded_test_X=padded_test_X.astype('int32')

In [ ]:
# These lines can be used instead of convert_and_pad and convert_and_pad_data functions under from keras.preprocessing.text import Tokenizer from keras.preprocessing.sequence import pad_sequences
# but I had to write my own function because of problems for deployment (conflict of libaries)
# # Tokenize the sentences
# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(list(train_X))

# train_X = tokenizer.texts_to_sequences(train_X)
# test_X = tokenizer.texts_to_sequences(test_X)

# ## Pad the sentences 
# train_X = pad_sequences(train_X, maxlen=maxlen)
# test_X = pad_sequences(test_X, maxlen=maxlen)

### Load Embeddings


In [ ]:
## FUNCTIONS TAKEN FROM https://www.kaggle.com/gmhost/gru-capsule

def load_glove(word_index):
    EMBEDDING_FILE = '../input/glove840b300d/glove.840B.300d.txt'    
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
# missing entries in the embedding are set using np.random.normal so we have to seed here too

if debug:
    embedding_matrix = np.random.randn(120000,300)
else:
    embedding_matrix = load_glove(vocab_to_int_train_x) #tokenizer.word_index

In [ ]:
np.savetxt('test1.txt', embedding_matrix)
b = np.loadtxt('test1.txt', dtype='float64')
b==embedding_matrix
# array([ True,  True,  True,  True], dtype=bool)

## Pytorch Model - TextCNN

In [ ]:
class CNN_Text(nn.Module):
    
    def __init__(self):
        super(CNN_Text, self).__init__()
        filter_sizes = [1,2,3,5]
        num_filters = 36
        n_classes = len(le.classes_) 
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.convs1 = nn.ModuleList([nn.Conv2d(1, num_filters, (K, embed_size)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(len(filter_sizes)*num_filters, n_classes)


    def forward(self, x):
        x = self.embedding(x)  
        x = x.unsqueeze(1)  
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] 
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  
        x = torch.cat(x, 1)
        x = self.dropout(x)  
        logit = self.fc1(x) 
        return logit

In [ ]:
embedding_matrix.shape

## Train TextCNN Model

In [ ]:
n_epochs = 50 #6
model = CNN_Text()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001) #lr=0.001
model.cuda()

# Load train and test in CUDA Memory
x_train = torch.tensor(padded_train_X, dtype=torch.long).cuda()
y_train = torch.tensor(train_y, dtype=torch.long).cuda()
x_cv = torch.tensor(padded_test_X, dtype=torch.long).cuda()
y_cv = torch.tensor(test_y, dtype=torch.long).cuda()

# Create Torch datasets
train = torch.utils.data.TensorDataset(x_train, y_train)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    start_time = time.time()
    # Set model to train configuration
    model.train()
    avg_loss = 0.  
    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x_batch)
        # Compute loss
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    # Set model to validation configuration -Doesn't get trained here
    model.eval()        
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv),len(le.classes_)))
    
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        # keep/store predictions
        val_preds[i * batch_size:(i+1) * batch_size] =F.softmax(y_pred).cpu().numpy()
    
    # Check Accuracy
    val_accuracy = sum(val_preds.argmax(axis=1)==test_y)/len(test_y)
    train_loss.append(avg_loss)
    valid_loss.append(avg_val_loss)
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))

In [ ]:
PATH="./textcnn_model1.pt"

In [ ]:
torch.save(model.state_dict(), PATH)

In [ ]:
model = CNN_Text()
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
device = "cuda:0"
model = model.to(device)

In [ ]:
def predict_single(df):  
    
    sentiment={0: 'neutral',1: 'negative', 2: 'positive'}
    
    df=df.apply(lambda x: CleanText(x).all_func())
    
#     words_as_int_train_x=words_as_int(vocab_to_int_train_x,df)

    x,_=convert_and_pad_data(vocab_to_int_train_x, df, pad=46)

    x = torch.tensor(x, dtype=torch.long).cuda()

    pred = model(x).detach()
    print(pred)

    pred = F.softmax(pred).cpu().numpy()
    print(pred)

    pred = pred.argmax(axis=1)

    pred = le.classes_[pred]
    

    return sentiment[pred[0]]

In [ ]:
x="U.S. GDP growth in fourth quarter raised slightly to 4.3% — and all signs point to economy speeding up"
df_test=pd.DataFrame([x])
df_test[0]

In [ ]:
# Sentiment prediction with CNN_Text model
predict_single(df_test[0])

In [ ]:
def plot_graph(epochs):
    fig = plt.figure(figsize=(12,12))
    plt.title("Train/Validation Loss")
    plt.plot(list(np.arange(epochs) + 1) , train_loss, label='train')
    plt.plot(list(np.arange(epochs) + 1), valid_loss, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('loss', fontsize=12)
    plt.legend(loc='best')

In [ ]:
plot_graph(n_epochs)

In [ ]:
import scikitplot as skplt
y_true = [le.classes_[x] for x in test_y]
y_pred = [le.classes_[x] for x in val_preds.argmax(axis=1)]
skplt.metrics.plot_confusion_matrix(
    y_true, 
    y_pred,
    figsize=(12,12),x_tick_rotation=90)

## Pytorch Model - BiLSTM

In [ ]:
class BiLSTM(nn.Module):
    
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.hidden_size = 64
        drp = 0.1
        n_classes = len(le.classes_)
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*4 , 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)


    def forward(self, x):
        #rint(x.size())
        h_embedding = self.embedding(x)
        #_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out

In [ ]:
n_epochs = 20
model = BiLSTM()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
model.cuda()

# Load train and test in CUDA Memory
x_train = torch.tensor(padded_train_X, dtype=torch.long).cuda()
y_train = torch.tensor(train_y, dtype=torch.long).cuda()
x_cv = torch.tensor(padded_test_X, dtype=torch.long).cuda()
y_cv = torch.tensor(test_y, dtype=torch.long).cuda()

# Create Torch datasets
train = torch.utils.data.TensorDataset(x_train, y_train)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    start_time = time.time()
    # Set model to train configuration
    model.train()
    avg_loss = 0.  
    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x_batch)
        # Compute loss
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    # Set model to validation configuration -Doesn't get trained here
    model.eval()        
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv),len(le.classes_)))
    
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        # keep/store predictions
        val_preds[i * batch_size:(i+1) * batch_size] =F.softmax(y_pred).cpu().numpy()
    
    # Check Accuracy
    val_accuracy = sum(val_preds.argmax(axis=1)==test_y)/len(test_y)
    train_loss.append(avg_loss)
    valid_loss.append(avg_val_loss)
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))

In [ ]:
PATH1="./BiLSTM.pt"

In [ ]:
torch.save(model.state_dict(), PATH1)

In [ ]:
# model = BiLSTM()
model.load_state_dict(torch.load(PATH1))
model.eval()

In [ ]:
plot_graph(n_epochs)

In [ ]:
# torch.save(model,'bilstm_model')

In [ ]:
import scikitplot as skplt
y_true = [le.classes_[x] for x in test_y]
y_pred = [le.classes_[x] for x in val_preds.argmax(axis=1)]
skplt.metrics.plot_confusion_matrix(
    y_true, 
    y_pred,
    figsize=(12,12),x_tick_rotation=90)

## Deploy : Predict A Single Example

In [ ]:
x="Dow Jones Futures Signals Modest Market Rally Retreat Amid Hedge Fund Fallout; Tesla, Nio Story Time Over"
df_test=pd.DataFrame([x])
df_test[0]

In [ ]:
device = "cuda:0"
model = model.to(device)

In [ ]:
# Sentiment prediction with BiLSTM model
predict_single(df_test[0])